In [1]:
import os
import torch
import zipfile
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from accelerate import init_empty_weights
from datasets import Dataset
 
# Set environment variable to manage memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
 
# Specify directories and the path to the zip file
offload_dir = os.path.expanduser("llama_offload/")
 
os.makedirs(offload_dir, exist_ok=True)
 
# Extract only the specified JSON file from the zip archive
target_file = "task024_cosmosqa_answer_generation.json"
 
# Load tokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer.pad_token = tokenizer.eos_token
 
# Load the model with accelerate's offloading and device map auto-setup
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Meta-Llama-3-8B",
        device_map="auto",
        offload_folder=offload_dir,
        load_in_8bit=True,
        llm_int8_enable_fp32_cpu_offload=True
    )
 
# Configure LoRA with reduced rank
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)
 



C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\accelerate\utils\modeling.py:841: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,

In [2]:
target_file = r"task024_cosmosqa_answer_generation.json"
with open(target_file, 'r', encoding='utf-8-sig') as f:
    json_data = json.load(f)

# Extract text data (assuming a structure where the data you want is under 'Instances')

instances = json_data['Instances'][0:2223]
input_texts = [str(instance['input']) for instance in instances]  # Convert to string if not already
output_texts = [str(instance['output'][0]) if instance['output'] else "" for instance in instances]  # Handle missing output
print(input_texts, output_texts)


["Context: Good Old War and person L : I saw both of these bands Wednesday night , and they both blew me away . seriously . Good Old War is acoustic and makes me smile . I really can not help but be happy when I listen to them ; I think it 's the fact that they seemed so happy themselves when they played . \nQuestion: In the future , will this person go to see other bands play ?", "Context: I mean it : not one person said ANYTHING to me . They would have if I did something wrong , right ? But , this morning , I got a call from my guy at the temp agency and he said that I was no longer needed in that position , that I did n't need to go out there . \nQuestion: Why might have the temp agency tell me I am not needed at that position ?", 'Context: Leaving my shift Thursday day shift I arrived the same time as my partner just after six that evening and before long the radio erupted in dispatch tones . A car fleeing the police has crashed and landed on its roof with four separate people entr

In [3]:
 
# Convert the texts into a Hugging Face Dataset
ds = Dataset.from_dict({'input': input_texts, 'output': output_texts})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["input"], examples["output"], truncation=True, padding="max_length", max_length=512)

# Apply tokenization and set format
tokenized_datasets = ds.map(tokenize_function, batched=True, remove_columns=["input", "output"])
tokenized_datasets.set_format("torch")

# Split dataset into train and eval
train_size = int(0.9 * len(tokenized_datasets))
train_dataset = tokenized_datasets.select(range(train_size))
eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))

# Define data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map: 100%|██████████| 2223/2223 [00:00<00:00, 5135.10 examples/s]


In [4]:
# Define training arguments
save_dir="finetuned-weights"
training_args = TrainingArguments(
    output_dir=save_dir,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    save_steps=1000,
    save_total_limit=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    report_to="none",
    fp16=torch.cuda.is_available(),
)
 
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)
 
# Train the model
trainer.train()
 
# Evaluate the model
with torch.no_grad():
    eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")
 
# Save the model and tokenizer

model.save_pretrained(os.path.join(save_dir, "fine-tuned-llama-lora"))
tokenizer.save_pretrained(os.path.join(save_dir, "fine-tuned-llama-lora"))
 
# Test the model on a sample input
input_text = "Hello, what is the meaning of life?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
 
with torch.no_grad():
    outputs = model.generate(inputs["input_ids"], max_length=50, num_return_sequences=1)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Generated output: {output_text}")
 
# Clear CUDA cache
torch.cuda.empty_cache()

C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\panshul2\AppData\Local\Temp\ipykernel_148716\2425887148.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/1500 [00:00<?, ?it/s]C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\llama\modeling_llama.py:602: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn

KeyboardInterrupt: 